In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys

# Add src/ to path for custom module imports
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root / "src"))

from paths import RAW_DATA_DIR

In [2]:
## read in the data
data_path = RAW_DATA_DIR / "upworthy-archive-exploratory-packages-03.12.2020.csv"
raw_data = pd.read_csv(data_path)
raw_data.head(1)

,Unnamed: 0,created_at,updated_at,clickability_test_id,excerpt,headline,lede,slug,eyecatcher_id,impressions,clicks,significance,first_place,winner,share_text,square,test_week
0,0,2014-11-20 06:43:16.005,2016-04-02 16:33:38.062,546d88fb84ad38b2ce000024,Things that matter. Pass 'em on.,They're Being Called 'Walmart's Worst Nightmar...,"<p>When I saw *why* people are calling them ""W...",theyre-being-called-walmarts-worst-nightmare-a...,546d6fa19ad54eec8d00002d,3052,150,100.0,True,True,Anyone who's ever felt guilty about shopping a...,NaN,201446


### First Impresions

#### Unuseful column
We have a column that needs to be dropped, that's 'Unnamed: 0' - it offers zero value.

#### Drop non-randomized tests
There is a known issue between 06-01-2013 and 01-31-2014. We need to drop those tests from the data because randomization is a requirement for any A/B test. Without it, we induce bias to the results and cannot reliably analyze the outputs. Fortunately, there is a column called *problem* that indivates if the test occured during this period. We can easily drop these.

#### Key column descriptions

Not every columns is useful for these tests, or a sort of meta-analysis

- *clickability_test_id*: Unique ID for each test. There are multiple rows per test.
- *headline*: this is the variable being tested. Not explicitly used in an A/B test, it's just the factor.
- *impressions*: denominator for a two-proportion z-test
- *clicks*: numerator for a two-proportion z-test
- *created_at*: this tells us when the factor was created
- *winner*: this tells us the ground truth winner - only useful for verification

The rest have more to do with online content than headlines. We want to test headlines only here.

### Key step: Make a data cleaning function

We will take this function and put it in a script when we finish, that way we can call it instead of relying on notebooks for key components of an analysis.

In [3]:
## always make a copy of the original for reference
df = raw_data.copy()

In [4]:
raw_data.columns

Index(['Unnamed: 0', 'created_at', 'updated_at', 'clickability_test_id',
       'excerpt', 'headline', 'lede', 'slug', 'eyecatcher_id', 'impressions',
       'clicks', 'significance', 'first_place', 'winner', 'share_text',
       'square', 'test_week'],
      dtype='object')

In [5]:
select_cols = ['clickability_test_id', 'created_at', 'headline', 'impressions', 'clicks','winner']

In [6]:
## select only the columns we need
print(raw_data.shape)
raw_data = raw_data[select_cols]
print(raw_data.shape)

(22666, 17)
(22666, 6)


In [7]:
raw_data.head()

,clickability_test_id,created_at,headline,impressions,clicks,winner
0,546d88fb84ad38b2ce000024,2014-11-20 06:43:16.005,They're Being Called 'Walmart's Worst Nightmar...,3052,150,True
1,546d88fb84ad38b2ce000024,2014-11-20 06:43:44.646,They're Being Called 'Walmart's Worst Nightmar...,3033,122,False
2,546d88fb84ad38b2ce000024,2014-11-20 06:44:59.804,They're Being Called 'Walmart's Worst Nightmar...,3092,110,False
3,546d902c26714c6c44000039,2014-11-20 06:54:36.335,This Is What Sexism Against Men Sounds Like,3526,90,False
4,546d902c26714c6c44000039,2014-11-20 06:54:57.878,This Is What Sexism Against Men Sounds Like,3506,120,False


In [8]:
raw_data['created_at'] = pd.to_datetime(raw_data['created_at'], format = 'mixed')

In [9]:
## remove non-randomized tests
raw_data = raw_data[~raw_data['created_at'].between("2013-06-01", "2014-01-31")].copy()
print(raw_data.shape)

(17547, 6)


Great, now let's add a proportion feature, rounded to 5 decimals, and a second table with tests and observation counts.

In [10]:
raw_data['proportion'] = raw_data['clicks'] / raw_data['impressions']

We are seeing 3,611 tests. 85 of them have two levels. Suggesting 3526 have 3+ levels. This is a bit unexpected, so we will investigate one of these multi-level tests.

In [11]:
counts = raw_data.value_counts('clickability_test_id').to_frame().reset_index()

In [12]:
counts[counts['count'] == counts['count'].max()]

,clickability_test_id,count
0,53480581be4154443a000008,14


In [13]:
counts[counts['count'] == 2]

,clickability_test_id,count
3526,533e2fecc383387fef000008,2
3527,54db6a6338666500280c0000,2
3528,532f40df995b7efb01000006,2
3529,5342d8b4df3e0e21c4000043,2
3530,541828d893f63351ec00003e,2
...,...,...
3606,53f63f7f3887042eaf000015,2
3607,5189a0d6d971067ca4001e18,2
3608,5470c9d40e78bd40ad000034,2
3609,54d19fde6466610019170000,2


In [14]:
pd.set_option('display.max_colwidth', None) 

This adds complexity. We are testing *headlines only* and these headlines are identical. As such, we need to rethink how we clean this data. It's obvious that there is one-level of headline data in this observation.

As such, it seems that we need to groupby and count the number of unique headlines per clickability_test_id. This many eliminate some tests, which is okay. We aren't interested in them anyways!

In [15]:
raw_data[raw_data['clickability_test_id'] == '53480581be4154443a000008']['headline']

12581    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12587    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12616    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12632    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12654    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12670    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12687    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12718    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12720    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDUM: Also, The Girls.
12759    This Dad Has A Message To The Boys Who Want To Date His Daughter. ADDENDU

We have some tests that have one headline - those need to be dropped from the dataset. We cannot test one headline against itself.

There are some that show multiple headline levels. We will need to test these factors, but first let's make sure these actually are unique headlines and not just random punctuation mistakes.

In [16]:
levels = raw_data.groupby(['clickability_test_id'])['headline'].nunique().to_frame().reset_index()
levels.sort_values(by = 'headline')

,clickability_test_id,headline
3610,55415ee3313766000c200000,1
1381,53d9e44345ba39cd440000c0,1
2752,54836027663239001e260000,1
1379,53d9399ffb557468d5000027,1
1378,53d92ec6fb5574a15d00002f,1
...,...,...
1523,53ed7f020d60d09e74000078,9
1560,53f42ff444df8303dd00001d,10
2406,5452940303b89045df000063,10
362,530803f8f3b21fc7e900002f,11


The data below show that these are different headlines. They're mentioning different famous people, which might change the number of impressions or clicks. I see the point here, but this feels a bit like over doing it. Were I developing this test, I'd discourage this. It's a test that is possible but it feels like someone is overthinking it.

Either way, we can test this data as-is.

In [17]:
raw_data[raw_data['clickability_test_id'] == '5308c3271e030e5930000003']['headline']

5379     Pink, Ellen, Katy Perry, And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5380        Ellen, Sofia Vergara, And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5381                   Ellen, A Rapper And Katy Perry Walk Into A Commercial And Destroy A Silly Argument
5382              Ellen, A Rapper And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5383                   Ellen, Janelle Monae, And Pink Walk Into A Commercial And Destroy A Silly Argument
5384        Ellen, Janelle Monae, And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5385                   Ellen, Pink, And Sofia Vergara Walk Into A Commercial And Destroy A Silly Argument
5386                 Ellen, Pink, And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5387        A Comedian, A Rapper, And A Hockey Player Walk Into A Commercial And Destroy A Silly Argument
5388                  Ellen, A Pop Star, And A

In [18]:
## use list notation to grab only the IDs with 2+ levels
tests = [levels['clickability_test_id'][i] for i in range(len(levels)) if levels['headline'][i] > 1]

As a result we get 1951 tests. We can work on custom logic to test each test by number of levels.

In [19]:
len(tests)

1951

It is possible that there are multiple lines containing impressions + clicks for the same headline. We need to check to see if we must groupby + sum these.

In [20]:
raw_data.shape

(17547, 7)

In [21]:
raw_data.groupby(['clickability_test_id','headline'], as_index = False)[['impressions', 'clicks']].sum()

,clickability_test_id,headline,impressions,clicks
0,51436061220cb800020001e7,Creationism Has Nothing To Do With Christianity According To Christian Dude,2551,39
1,51436061220cb800020001e7,"Creationism Shouldn't Be Taught In Science Classes, According To This Guy.",2539,49
2,51436061220cb800020001e7,God Finds Out About Creationism And Sends A Representative To Put A Stop To It,2661,63
3,51436061220cb800020001e7,The One Where A Creationist Picks A Fight And Loses To A Priest,2629,68
4,51436069220cb800020005ae,"The Advertisements You Read Every Day, Only Naked",1986,44
...,...,...,...,...
10296,55403be0393131002cf60000,"When we're done with plastic, we send it on quite a journey. One that may make its way back to us.<br>",1983,4
10297,55413ef4333531000c180000,"They spent 4 days in the swamp looking for a creature many thought extinct. On day 5, they found it.",2091,32
10298,55413ef4333531000c180000,"They spent 4 days in the swamp looking for a monkey many thought extinct. On day 5, they found it.",2014,18
10299,554141c4383063001c110000,Sunshiny Vitamin D costs zero dollars and magically turns cholesterol into calcium,10140,24


The shape decreased, therefore we did have duplicate lines for headlines. We need to add this as a final step in our data cleaning.

Lets get all of this into a function.

In [22]:
def clean_data(df):
    ### identify columns of interest
    select_cols = ['clickability_test_id', 'created_at', 'headline', 'impressions', 'clicks','winner']
    
    ### select only these columns
    df = df[select_cols]

    ### convert to datetime
    df['created_at'] = pd.to_datetime(df['created_at'], format = 'mixed')

    ### filter out bad experiments due to non-randomization
    df = df[~df['created_at'].between("2013-06-01", "2014-01-31")]

    ### identify unique number of headlines tested per test
    levels = df.groupby(['clickability_test_id'])['headline'].nunique().to_frame().reset_index()

    ### identify the 2+ tests
    tests = [levels['clickability_test_id'][i] for i in range(len(levels)) if levels['headline'][i] > 1]

    ### select only the possible tests
    df = df[df['clickability_test_id'].isin(tests)]

    ### group the data to combine impressions and clicks at the headline level
    dfgrouped = df.groupby(['clickability_test_id','headline'], as_index = False)[['impressions', 'clicks']].sum()

    ### create the proportion column for any two-prop z-tests
    dfgrouped['proportion'] = dfgrouped['clicks'] / dfgrouped['impressions']

    return dfgrouped
    


In [23]:
output = clean_data(df)
print(output['clickability_test_id'].nunique())
output.head()

1951


<positron-console-cell-23>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,clickability_test_id,headline,impressions,clicks,proportion
0,51436061220cb800020001e7,Creationism Has Nothing To Do With Christianity According To Christian Dude,2551,39,0.015288
1,51436061220cb800020001e7,"Creationism Shouldn't Be Taught In Science Classes, According To This Guy.",2539,49,0.019299
2,51436061220cb800020001e7,God Finds Out About Creationism And Sends A Representative To Put A Stop To It,2661,63,0.023675
3,51436061220cb800020001e7,The One Where A Creationist Picks A Fight And Loses To A Priest,2629,68,0.025865
4,51436069220cb800020005ae,"The Advertisements You Read Every Day, Only Naked",1986,44,0.022155


### Cleaning takeaways

- The owners of the datset were testing more than just headlines. These could have been multivariate tests (headline + lede/slug interaction) or univariate on a different level. That's different than what we're doing here, but okay.

- The headlines are extremely clickbait-y

- We need to account for the multiple levels of this test